In [34]:
import pandas as pd
import numpy as np


In [36]:
# для всех норм по ТПГ нужно объявить эту переменную
row_number = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,usecols=[5])

skip_row = row_number.ЦОК.tolist()
skip_row_n = skip_row.index('подгруппа')-1

In [10]:
#СЧИТАЕТ КОЛИЧЕСТВО ОТКРЫТЫХ ЗАКРЫТЫХ (всех)

# Загружаем нужные столбцы
cols = ['Номер из NAV','Адрес магазина','дата планового открытия','статус','ИФ кольца','ИФ КОЛЬЦА ОБРУЧ',
        'ИФ печатки','ИФ серьги','ИФ подвес культ','ИФ подвес декор','БК кольца','БК кольца обруч','БК печатки',
        'БК серьги','БК подвес культ','БК подвес декор','ИТОГО, нормы в штуках, без накоплений']

df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=cols)

# Убираем ненужную инфу
df.rename({'ИТОГО, нормы в штуках, без накоплений':"sum_all"}, axis=1, inplace=True)
df = df[df['sum_all'] > 0]
df['статус'] = df.статус.str.lower()
df = df[df.статус.isin(['открыт','склад','отгружено'])]
df['Номер из NAV'] != 4801 #откроется через год

# Добавляем сумму по ИФ\БК и оставляем этот столбец вместо столбцов с нормами по иф\бк
sum_cols = [x for x in cols if 'ИФ' in x or 'БК' in x ]
df['sum_gold'] = df[sum_cols].sum(axis=1)
df.drop(axis=1, columns=sum_cols, inplace=True)

# Количество открытых и к открытию всего и только по золоту
shops_count = pd.DataFrame({
    'Магов открыто': [df.sum_all[df.статус =='открыт'].count(), df.sum_gold[(df.sum_gold > 0) & (df.статус =='открыт')].count()],
    'Магов к открытию': [df.sum_all[df.статус !='открыт'].count(), df.sum_gold[(df.sum_gold > 0) & (df.статус !='открыт')].count()],
    'Итого магов': [df.sum_all.count(),df.sum_gold[df.sum_gold > 0].count()]})

# Список новых магов с датами
new_shops = df[['Номер из NAV', 'Адрес магазина', 'дата планового открытия']][(df.sum_gold > 0) & (df.статус !='открыт')]
new_shops.sort_values('дата планового открытия', inplace=True)

# Нормы ИФ/БК по ТПГ
norm_all = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12',skiprows=4,nrows=skip_row_n)

list = norm_all.columns.tolist()

cols = (
    [list[list.index('Название из NAV')]] +
    list[list.index(1001):list.index(1102)+1] +
    list[list.index(1501):list.index(1597)+1]
)
norm_all = norm_all[cols]
gold_shops = df[['Номер из NAV', 'статус']][df.sum_gold > 0]
merged = pd.merge(gold_shops, norm_all, left_on='Номер из NAV', right_on='Название из NAV', how = 'left')

del merged['Название из NAV'], merged['Номер из NAV']

merged = merged.groupby(by='статус').sum()
merged = merged.T
merged['sum_all'] = merged.sum(axis=1)
merged = merged[['открыт', 'sum_all']]

# СОХРАНЯЕМ ВСЕ В 1 ФАЙЛ

with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\нужное\для_новые_нормы.xlsx'
) as writer:
    shops_count.to_excel(writer, sheet_name='количество')
    new_shops.to_excel(writer, sheet_name='новые_маги', index=False)
    merged.to_excel(writer, sheet_name='новые_нормы')

In [14]:
#ДОСТАЕТ НОРМЫ ПО БУКВАМ И ЗОДИАКАМ 

# Читаем реестр магов
norm = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='НОРМЫ 17.12', skiprows=4, nrows=skip_row_n)

norm.set_index('Название из NAV',inplace=True)

b_columns = [1088, 1089, 1090, 1586, 1587, 1588]
z_columns = [1091, 1092, 1093, 1589, 1590, 1591]

norm_b = norm[b_columns]
norm_z = norm[z_columns]

norm_b = norm_b.assign(sum_if = norm_b[1088] + norm_b[1089] + norm_b[1090],
              sum_bk = norm_b[1586] + norm_b[1587] + norm_b[1588])

norm_z = norm_z.assign(sum_if1 = norm_z[1091] + norm_z[1092] + norm_z[1093],
              sum_bk1 = norm_z[1589] + norm_z[1590] + norm_z[1591])

norm_z = norm_z[norm_z.columns[[0,1,2,6,3,4,5,7]]]

with pd.ExcelWriter (r'C:\Остатки\norm_bz.xlsx') as writer:
    norm_b.to_excel(writer, sheet_name='norm_b')
    norm_z.to_excel(writer, sheet_name='norm_z')

In [37]:
#ВЫДАЕТ СПИСОК АКТУАЛЬНЫХ МАГОВ ИФ\БК С НОРМАМИ ПО ТГ И ДАТЫ ОТКРЫТИЯ

#выгружаем датафрэйм с актуальными нормами ИФ БК из реестра магов 
cols_df = ['Номер из NAV', 'Адрес магазина', 'дата планового открытия','статус', 'ИФ кольца', 'ИФ КОЛЬЦА ОБРУЧ', 'ИФ печатки', 'ИФ серьги', 
        'ИФ подвес культ', 'ИФ подвес декор', 'БК кольца', 'БК кольца обруч', 'БК печатки', 'БК серьги', 
        'БК подвес культ', 'БК подвес декор']
      
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=cols_df)

#выбираем открытые маги с суммой норм по иф/бк больше нуля
df = df[df.статус.str.contains('откр|отгр|склад', na=False, case=False)]
df['сумма'] = df.iloc[:,3:].sum(axis=1)
df = df[df['сумма'] > 0]
df = df[df['Номер из NAV'] != 3323]# Тамбов носовская


all_shops = df.drop(columns=['статус','сумма','дата планового открытия'])
all_shops = all_shops.rename(columns={'Номер из NAV':'Код'})
all_shops['Код'] = all_shops['Код'].astype(np.int64)

# делим на два отдельных иф и бк
cols = all_shops.columns.tolist()
shops_if = all_shops[[*cols[:5],cols[7],cols[6],cols[5]]]
shops_bk = all_shops[[*cols[:2], *cols[8:11], cols[13],cols[12],cols[11]]]

#выгружаем из файла "наполняемость" актуальную структуру магов иф/бк, добавляет туда нормы и новые маги из датафрэйма выше
shops_if_list = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='ИФ',skiprows=3, usecols=[2],keep_default_na=False)

shops_bk_list = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\наполняемость с 10012019.xlsx',
    sheet_name='БК',skiprows=3, usecols=[2],keep_default_na=False)

# список новых магов которые надо добавить
add_to_if = pd.DataFrame({'Код':[x for x in shops_if['Код'].tolist() if x not in shops_if_list['Код'].tolist()]})
add_to_bk = pd.DataFrame({'Код':[x for x in shops_bk['Код'].tolist() if x not in shops_bk_list['Код'].tolist()]})

# добавляем новые маги в структуру
shops_if_list = shops_if_list.append(add_to_if, sort=False)
shops_bk_list = shops_bk_list.append(add_to_bk, sort=False)

#соединяем "структуру" и нормы
shops_if = pd.merge(shops_if_list,shops_if,how='left',on='Код')
shops_bk = pd.merge(shops_bk_list,shops_bk,how='left',on='Код')


#удаляем индекс и сохраняем на отдельные вкладки
shops_if.set_index('Код', drop=True,inplace=True)
shops_bk.set_index('Код', drop=True,inplace=True)

#создаем датафрейм из магов с датами открытия последние 90 дней
from datetime import date, timedelta

three_month_ago = date.today() - timedelta(90)
df['дата планового открытия'] = df['дата планового открытия'].apply(lambda x:x.date())#чтобы ошибка не выпадала о формате
dates = df[['Номер из NAV', 'дата планового открытия']][df['дата планового открытия'] > three_month_ago]
dates.set_index('Номер из NAV', drop=True,inplace=True)

#сохраняем все в эксель
with pd.ExcelWriter (
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Семен\актуальные маги.xlsx'
) as writer:
    shops_if.to_excel(writer, sheet_name='ИФ')
    shops_bk.to_excel(writer, sheet_name='БК')
    dates.to_excel(writer, sheet_name='даты')

In [33]:
# СПИСОК НОРМ ПО ТГ

#читает файл с реестром
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2)

#вынимает нужные две строки и сохраняет
rep_tn = df.iloc[0:1,28:74]
rep_tn.to_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\Отдел закупки\ЛИЧНЫЕ\Татьяна ИФ.БК\РЕЙТИНГИ\Рейтинг ТГ и ТН\123.xlsx',
index=False)

NameError: name 'pd' is not defined

In [3]:
#СЧИТАЕТ КОЛИЧЕСТВО МАГАЗОВ 3 МЕСЯЦА ДО СЕГОДНЯ И ПОСЛЕ
df = pd.read_excel(
    r'\\gold585.int\uk\Общее хранилище файлов\Коммерческий департамент\реестр магазинов\РЕЕСТР МАГАЗИНОВ NEW.xlsm',
    sheet_name='БАЗА',skiprows=2, usecols=([2, 3, 6, 18] + list(range(28, 41))), parse_dates=[6] )

# выбираем маги с нужным статусом, с нормами по тн больше нуля 
df['статус'] = df.статус.str.lower()
df = df[df.статус.isin(['открыт','склад','отгружено'])]
df[['дата планового открытия']] = df['дата планового открытия'].fillna(df['дата планового открытия'].min())
df['sum'] = df.iloc[ : , 5:].sum(axis=1)
df = df[df['sum']>0]


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 955 entries, 1 to 1846
Data columns (total 18 columns):
Номер из NAV               955 non-null float64
Адрес магазина             955 non-null object
дата планового открытия    955 non-null datetime64[ns]
статус                     955 non-null object
ИФ кольца                  955 non-null float64
ИФ КОЛЬЦА ОБРУЧ            955 non-null float64
ИФ печатки                 955 non-null float64
ИФ серьги                  955 non-null float64
ИФ подвес культ            955 non-null float64
ИФ подвес декор            955 non-null float64
ИФ аксессуары              955 non-null float64
БК кольца                  955 non-null float64
БК кольца обруч            955 non-null float64
БК печатки                 955 non-null float64
БК серьги                  955 non-null float64
БК подвес культ            955 non-null float64
БК подвес декор            955 non-null float64
sum                        955 non-null float64
dtypes: datetime64[ns](1)

In [ ]:
from ipywidgets import DatePicker
DPicker = DatePicker()
display(DPicker)

def date_value(date):
    
    return date

DPicker.observe(date_value, names='value')
print(date)

In [4]:
dPicker.observe()

TypeError: observe() missing 1 required positional argument: 'handler'

In [1]:
from IPython.display import display
from ipywidgets import DatePicker

dPicker = DatePicker()

def on_button_clicked(b):
    print(b['old'], '->', b['new'])

dPicker.observe(on_button_clicked, names='value')
display(dPicker)

DatePicker(value=None)

None -> 2024-07-02


In [23]:
from IPython.display import display
from ipywidgets import DatePicker
D = DatePicker()
display(D)
def e(change):
    date = change.new
D.observe(e,'value')
print(date)

DatePicker(value=None)

2017-05-26


In [20]:

date

datetime.date(2017, 5, 26)

In [10]:
print(date)

None


In [8]:

df[['дата планового открытия']][df['дата планового открытия'] >= datetime.strptime('2020-01-01','%Y-%m-%d')]
# тут надо pd.todate

,дата планового открытия
952,2020-01-18
953,2020-01-15
954,2020-02-01
955,2020-02-01
956,2020-02-01
957,2020-02-01
958,2020-02-01
959,2020-02-01
960,2020-02-08
962,2020-02-14


In [3]:
from datetime import date
date.today().month + 12 - 3


12

In [52]:
def threem_ago():
    if date.today().month < 4:
        return date.today().month + 12 - 3
    else:
        return date.today().month - 3

    

In [53]:
threem_ago()

10

In [74]:
date(2019-10-1)

SyntaxError: invalid token (<ipython-input-74-36842a70b676>, line 1)

In [81]:
datetime.strptime('2007-03-16','%Y-%m-%d')

datetime.datetime(2007, 3, 16, 0, 0)

In [1]:
x_list = [5,3,7,8,4,1,2]
[*x_list[:3],x_list[3]]

[5, 3, 7, 8]

In [37]:
shops_if = all_shops[*cols[:5], cols[7],cols[6],cols[5]]


SyntaxError: can't use starred expression here (<ipython-input-37-b21b6f1c9de7>, line 1)

In [2]:
import pandas as pd
pd.read_html('http://www.profinance.ru/currency_eur,asp')[9].at[3,1]

'87.057 / 87.077'

In [5]:
!pip install yfinance --user

ERROR: Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: 'C:\\Users\\DOTSEN~1.SEM\\AppData\\Local\\Temp\\pip-unpack-7w4p0e08\\yfinance-0.1.54.tar.gz'
Check the permissions.

You should consider upgrading via the 'python -m pip install --upgrade pip' command.
